In [ ]:
%load_ext autoreload
%autoreload 2
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
from scipy.integrate import solve_ivp
from scipy.integrate._ivp.rk import RK45, RK23, DOP853
import time

# Add the parent directory to sys.path to import invflow modules
parent_dir = os.path.dirname(os.getcwd())
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

# Import from dputils
from dputils import double_pendulum
from homproj.adaptive import create_projected_solver

# Set up the double pendulum problem
f_pendulum, invariants, variables = double_pendulum()
H_sym = invariants[0]
print(H_sym)
H_fun = sp.lambdify(variables, H_sym, 'numpy')

def compute_energy(state):
    if state.ndim == 1:
        return H_fun(*state)
    else:
        return np.array([H_fun(*row) for row in state])

def f(t, y):
    return f_pendulum(y)

In [ ]:
def phi(x):
    x1,x2,x3,x4 = x
    c12 = np.cos(x1 - x2)
    D   = np.sqrt(np.maximum(0.0, 2.0 - c12*c12))
    y1  = -2.0*np.cos(x1)
    y2  = -1.0*np.cos(x2)
    y3  =  0.5*x3
    y4  = (2.0*x4 - c12*x3)/(2.0*D)
    return np.array([y1,y2,y3,y4])

def _acos_branch(val, prev=None):
    base = np.arccos(np.clip(val, -1.0, 1.0))   # in [0, pi]
    if prev is None: return base
    # pick branch closest to prev
    cands = np.array([ base, -base ])
    cands += 2*np.pi*np.round((prev - cands)/(2*np.pi))
    return cands[np.argmin(np.abs(cands - prev))]

def phi_inv(y, x_prev=None):
    y1,y2,y3,y4 = y
    x1 = _acos_branch(-y1/2.0, x_prev[0] if x_prev is not None else None)
    x2 = _acos_branch(-y2,      x_prev[1] if x_prev is not None else None)
    x3 = 2.0*y3
    c12 = np.cos(x1 - x2)
    D   = np.sqrt(np.maximum(0.0, 2.0 - c12*c12))
    x4  = 0.5*c12*x3 + D*y4
    return np.array([x1,x2,x3,x4])

In [ ]:
from homproj.linhomproj import LinearHomogeneousProjector
from homproj import HomogeneousProjector
from homproj import Projector
from homproj.numeric import gll4, gll6, pseudo_symplectic, rk4


def init_solver_methods(initial_state):
    pseudo_hom_proj = HomogeneousProjector(
        invariants=invariants[0],
        variables=variables,
        initial_state=initial_state,
        integrator='rk2',
        max_iterations=2,
    )

    conj_hom_proj = LinearHomogeneousProjector(
        invariant=H_fun,
        generator=np.array([1, 1, 1/2, 1/2]),
        degree=1,
        initial_state=initial_state,
        phi=phi,
        phi_inv=phi_inv,
    )

    rk4_proj = lambda x, h, f: pseudo_hom_proj.project(rk4(x, h, f))
    rk4_proj_c = lambda x, h, f: conj_hom_proj.project(rk4(x, h, f))

    # List of (label, method) pairs to test
    solver_methods = [
        ('RK4', RK45),
        # ('CH Projection', create_projected_solver(RK45, conj_hom_proj)),
        # ('PNH Projection', create_projected_solver(RK45, pseudo_hom_proj)),
        ('RK4', rk4), # fixed step size
        ('CH RK4', rk4_proj_c), # fixed step size
        ('PNH RK4', rk4_proj), # fixed step size
        ('PS4(9)', pseudo_symplectic),
        ('GLL4', gll4),
        ('Reference', DOP853),
    ]
    return solver_methods


def init_adaptive_solver_methods(initial_state):
    pseudo_hom_proj = HomogeneousProjector(
        invariants=H_fun,
        initial_state=initial_state,
        integrator='rk2',
        max_iterations=1,
    )

    conj_hom_proj = LinearHomogeneousProjector(
        invariant=H_fun,
        generator=np.array([1, 1, 1/2, 1/2]),
        degree=1,
        initial_state=initial_state,
        phi=phi,
        phi_inv=phi_inv,
    )

    rk4_proj = lambda x, h, f: pseudo_hom_proj.project(rk4(x, h, f))
    rk4_proj_c = lambda x, h, f: conj_hom_proj.project(rk4(x, h, f))

    # List of (label, method) pairs to test
    solver_methods = [
        ('RK45', RK45),
        # ('CH Projection', create_projected_solver(RK45, conj_hom_proj)),
        ('PNH Projection', create_projected_solver(RK45, pseudo_hom_proj)),
        # ('RK4', rk4), # fixed step size
        # ('CH RK4', rk4_proj_c), # fixed step size
        # ('PNH RK4', rk4_proj), # fixed step size
        # ('PS4(9)', pseudo_symplectic),
        # ('GLL4', gll4),
        ('GLL6', gll6),
        ('Reference', DOP853),
    ]
    return solver_methods



In [ ]:
from dputils import plot_trajectories, plot_analysis_over_initial_conditions, run_experiment

np.random.seed(2)

n_runs = 10
tmax=100
tol=1e-8
timestep=0.05

solutions_list = []
timings_list = []
initial_energy_list = []

def run_single_experiment(i, tmax, tol, timestep):
    # Set up initial state with random second angle
    a = np.random.uniform(-0.1, 0.1)
    b = np.random.uniform(-0.1, 0.1)
    y0 = np.array([a, b, 1, -1])

    initial_energy = compute_energy(y0)
    print(initial_energy)
    # solver_methods = init_solver_methods(y0)
    solver_methods = init_adaptive_solver_methods(y0)
    solutions, timings = run_experiment(solver_methods, y0, tmax, tol, timestep, f)
    return solutions, timings, initial_energy

for i in range(n_runs):
    print(i)
    solutions, timings, initial_energy = run_single_experiment(i, tmax=tmax, tol=tol, timestep=timestep)

    fig_combined = plot_analysis_over_initial_conditions(
        solutions_list=[solutions],
        compute_energy=compute_energy,
        initial_energies=[initial_energy],
        timings=[timings],
    )
    plt.show()

    solutions_list.append(solutions)
    timings_list.append(timings)
    initial_energy_list.append(initial_energy)

print(f"Completed {n_runs} experiments with different initial conditions")



In [ ]:
# Create trajectory visualizations
fig_traj = plot_trajectories(
    solutions=solutions,
    start_time=80,
    end_time=100,
)
plt.show()

# fig_traj.savefig('../figs/double_pendulum_40-60s_window_2.png', dpi=300, bbox_inches='tight')


In [ ]:
# Create time series, energy and performance visualizations in a single figure
# For a single initial condition, wrap solutions in a list
fig_combined = plot_analysis_over_initial_conditions(
    solutions_list=solutions_list,
    compute_energy=compute_energy,
    initial_energies=initial_energy_list,
    timings=timings_list,
)
plt.show()

fig_combined.savefig('../figs/double_pendulum_analysis_adaptive.png', dpi=300, bbox_inches='tight')


In [ ]:
# Create an animated GIF of the double pendulum simulation
from dputils import create_pendulum_animation
if False:
    animation = create_pendulum_animation(
        solutions=solutions,
        compute_energy=compute_energy,
        initial_energy=initial_energy,
        filename='double_pendulum_energy_preservation.gif',
        fps=20,
        time_step=0.05
    )

    # Display the animation in the notebook (this may not work in all environments)
    # If it doesn't work, you can open the saved GIF file directly
    from IPython.display import Image
    try:
        display(Image(filename='double_pendulum_energy_preservation.gif'))
        print("Animation displayed successfully!")
    except (IOError, RuntimeError, ValueError) as e:
        print(f"Animation saved but couldn't display it in the notebook: {e}")
        print("Please open 'double_pendulum_energy_preservation.gif' to view the animation.")